In [1]:
import pandas as pd
import numpy as np
import time
import math
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.ensemble import AdaBoostClassifier as AdaBoost
from sklearn.ensemble import BaggingClassifier as Bagging
from sklearn.ensemble import GradientBoostingClassifier as GBoost
from sklearn.ensemble import VotingClassifier as Voting
from sklearn import preprocessing
import xgboost as xgb

/Users/vol/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/vol/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Do training data
### Load amd check training dataset

In [2]:
# load data
train = pd.read_csv("train.csv")
print train.shape

(118529, 17)


In [3]:
train.head()

,voted,gender,cd,hd,age,dbdistance,vccdistance,party,racename,hsonly,mrrg,chldprsnt,cath,evang,nonchrst,otherchrst,days.since.reg
0,Y,M,7.0,31.0,36,NaN,NaN,U,Hispanic,25.4,63.4,54.0,16.7,16.5,39.6,27.3,420
1,Y,F,6.0,38.0,55,NaN,NaN,U,Uncoded,7.9,97.8,59.8,16.7,15.5,30.9,36.9,307
2,Y,F,2.0,53.0,24,NaN,NaN,U,Caucasian,50.2,7.6,49.5,14.6,24.0,29.6,31.7,292
3,Y,F,7.0,30.0,25,NaN,NaN,D,Caucasian,38.0,8.5,47.4,13.1,22.3,33.3,31.4,316
4,Y,M,5.0,19.0,22,NaN,NaN,R,Caucasian,30.5,19.1,23.1,16.0,10.5,39.1,34.5,392


### Check and fill missing data

In [4]:
# check NaN column by column
print 'gender any null? ' + str(train['gender'].isnull().values.any())
print 'cd any null? ' + str(train['cd'].isnull().values.any())
print 'hd any null? ' + str(train['hd'].isnull().values.any())
print 'age any null? ' + str(train['age'].isnull().values.any())
print 'dbdistance any null? ' + str(train['dbdistance'].isnull().values.any())
print 'vccdistance any null? ' + str(train['vccdistance'].isnull().values.any())
print 'party any null? ' + str(train['party'].isnull().values.any())
print 'racename any null? ' + str(train['racename'].isnull().values.any())
print 'hsonly any null? ' + str(train['hsonly'].isnull().values.any())
print 'mrrg any null? ' + str(train['mrrg'].isnull().values.any())
print 'chldprsnt any null? ' + str(train['chldprsnt'].isnull().values.any())
print 'cath any null? ' + str(train['cath'].isnull().values.any())
print 'evang any null? ' + str(train['evang'].isnull().values.any())
print 'nonchrst any null? ' + str(train['nonchrst'].isnull().values.any())
print 'otherchrst any null? ' + str(train['otherchrst'].isnull().values.any())
print 'days.since.reg any null? ' + str(train['days.since.reg'].isnull().values.any())
#print 'Id any null? ' + str(train['Id'].isnull().values.any())

gender any null? False
cd any null? True
hd any null? True
age any null? False
dbdistance any null? True
vccdistance any null? True
party any null? False
racename any null? False
hsonly any null? False
mrrg any null? False
chldprsnt any null? False
cath any null? False
evang any null? False
nonchrst any null? False
otherchrst any null? False
days.since.reg any null? False


In [5]:
print train['dbdistance'].mean()
print train['dbdistance'].median()
print train['vccdistance'].mean()
print train['vccdistance'].median()

print train['cd'].mean()
print train['cd'].median()
print train['hd'].mean()
print train['hd'].median()

2.91094259371
2.27622
3.24225375615
2.444945
3.76732727564
4.0
30.9323782767
29.0


In [6]:
# plt.plot(train['dbdistance'])
# plt.show()
# plt.plot(train['vccdistance'])
# plt.show()
# plt.plot(train['voted'])
# plt.show()

In [7]:
# fill missing data
predictors_na = ['cd', 'hd', 'dbdistance', 'vccdistance']
train_filled = pd.DataFrame(train)
train_filled[predictors_na] = train_filled[predictors_na].apply(lambda x:x.fillna(x.median()))


In [8]:
print 'cd any null? ' + str(train_filled['cd'].isnull().values.any())
print 'hd any null? ' + str(train_filled['hd'].isnull().values.any())
print 'dbdistance any null? ' + str(train_filled['dbdistance'].isnull().values.any())
print 'vccdistance any null? ' + str(train_filled['vccdistance'].isnull().values.any())

cd any null? False
hd any null? False
dbdistance any null? False
vccdistance any null? False


### transformate parameters

In [9]:
train_filled['newage'] = np.log(train_filled['age'])
# train_filled['newage'] = train_filled['age']

train_filled['newchldprsnt'] = np.sqrt(train_filled['chldprsnt'])
# train_filled['newchldprsnt'] = train_filled['chldprsnt']


In [10]:
# import matplotlib.pyplot as plt
# plt.plot(train_filled['dbdistance'])
# plt.show()

### Delete parameters

### Do one-hot encoding and split my_train and my_test

In [11]:
# one-hot encoding categorical predictors
predictors = ['gender', 'cd', 'hd', 'newage', 'dbdistance', 'vccdistance',
       'party', 'racename', 'hsonly', 'mrrg', 'newchldprsnt', 'cath', 'evang',
       'nonchrst', 'otherchrst', 'days.since.reg']
cate = ['gender', 'cd', 'hd', 'party', 'racename', ]

# predictors = ['gender', 'cd', 'hd', 'newage','party', 'racename', 
#               'hsonly', 'mrrg', 'newchldprsnt','evang',
#               'nonchrst', 'otherchrst', 'days.since.reg']
# cate = ['gender', 'cd', 'hd', 'party', 'racename', ]

df_x=pd.DataFrame(train_filled[predictors])
df_x.head()

,gender,cd,hd,newage,dbdistance,vccdistance,party,racename,hsonly,mrrg,newchldprsnt,cath,evang,nonchrst,otherchrst,days.since.reg
0,M,7.0,31.0,3.583519,2.27622,2.444945,U,Hispanic,25.4,63.4,7.348469,16.7,16.5,39.6,27.3,420
1,F,6.0,38.0,4.007333,2.27622,2.444945,U,Uncoded,7.9,97.8,7.733046,16.7,15.5,30.9,36.9,307
2,F,2.0,53.0,3.178054,2.27622,2.444945,U,Caucasian,50.2,7.6,7.035624,14.6,24.0,29.6,31.7,292
3,F,7.0,30.0,3.218876,2.27622,2.444945,D,Caucasian,38.0,8.5,6.884766,13.1,22.3,33.3,31.4,316
4,M,5.0,19.0,3.091042,2.27622,2.444945,R,Caucasian,30.5,19.1,4.806246,16.0,10.5,39.1,34.5,392


In [12]:
# one hot encoding 'gender'
onehot = pd.get_dummies(df_x['gender'],prefix='gen')
df_x = df_x.drop('gender', axis=1)
df_x = df_x.join(onehot)

# one hot encoding 'cd'
onehot = pd.get_dummies(df_x['cd'],prefix='cd')
df_x = df_x.drop('cd', axis=1)
df_x = df_x.join(onehot)

# one hot encoding 'hd'
onehot = pd.get_dummies(df_x['hd'],prefix='hd')
df_x = df_x.drop('hd', axis=1)
df_x = df_x.join(onehot)

# one hot encoding 'party'
onehot = pd.get_dummies(df_x['party'],prefix='pty')
df_x = df_x.drop('party', axis=1)
df_x = df_x.join(onehot)

# one hot encoding 'racename'
onehot = pd.get_dummies(df_x['racename'])
df_x = df_x.drop('racename', axis=1)
df_x = df_x.join(onehot)

df_x.head(7)
print df_x.shape

(118529, 103)


In [13]:
# separate x, y
x=df_x.values
y=train_filled['voted'].values
# print x[0:3,:]
# print y[0:3]

In [14]:
# Randomly split into train and test sets
my_x_train, my_x_test, my_y_train, my_y_test = train_test_split(x, y, test_size=0.25, random_state=42)

## Do testing data
### Load testing data

In [15]:
# load data
test = pd.read_csv("test.csv")
print test.shape
test.head()

(39510, 17)


,gender,cd,hd,age,dbdistance,vccdistance,party,racename,hsonly,mrrg,chldprsnt,cath,evang,nonchrst,otherchrst,days.since.reg,Id
0,M,2.0,52.0,30,NaN,NaN,L,Caucasian,19.5,21.2,25.3,9.8,16.6,45.2,28.4,393,1
1,F,5.0,19.0,20,NaN,NaN,U,Caucasian,39.7,20.2,29.1,12.0,14.4,41.4,32.2,668,2
2,M,4.0,44.0,56,NaN,NaN,R,Caucasian,11.3,62.7,41.3,14.8,14.7,36.0,34.6,606,3
3,F,7.0,34.0,20,NaN,NaN,R,Caucasian,32.8,11.6,33.1,14.5,10.3,44.6,30.6,565,4
4,F,6.0,41.0,26,NaN,NaN,D,Uncoded,10.2,14.7,22.4,8.2,18.4,43.5,29.9,336,5


### Check and fill missing data

In [16]:
# check NaN column by column
print 'gender any null? ' + str(test['gender'].isnull().values.any())
print 'cd any null? ' + str(test['cd'].isnull().values.any())
print 'hd any null? ' + str(test['hd'].isnull().values.any())
print 'age any null? ' + str(test['age'].isnull().values.any())
print 'dbdistance any null? ' + str(test['dbdistance'].isnull().values.any())
print 'vccdistance any null? ' + str(test['vccdistance'].isnull().values.any())
print 'party any null? ' + str(test['party'].isnull().values.any())
print 'racename any null? ' + str(test['racename'].isnull().values.any())
print 'hsonly any null? ' + str(test['hsonly'].isnull().values.any())
print 'mrrg any null? ' + str(test['mrrg'].isnull().values.any())
print 'chldprsnt any null? ' + str(test['chldprsnt'].isnull().values.any())
print 'cath any null? ' + str(test['cath'].isnull().values.any())
print 'evang any null? ' + str(test['evang'].isnull().values.any())
print 'nonchrst any null? ' + str(test['nonchrst'].isnull().values.any())
print 'otherchrst any null? ' + str(test['otherchrst'].isnull().values.any())
print 'days.since.reg any null? ' + str(test['days.since.reg'].isnull().values.any())
print 'Id any null? ' + str(test['Id'].isnull().values.any())

gender any null? False
cd any null? True
hd any null? True
age any null? False
dbdistance any null? True
vccdistance any null? True
party any null? False
racename any null? False
hsonly any null? False
mrrg any null? False
chldprsnt any null? False
cath any null? False
evang any null? False
nonchrst any null? False
otherchrst any null? False
days.since.reg any null? False
Id any null? False


In [17]:
# fill missing data
predictors_na = ['cd', 'hd', 'dbdistance', 'vccdistance']
test_filled = pd.DataFrame(test)
test_filled[predictors_na] = test_filled[predictors_na].apply(lambda x:x.fillna(x.median()))


### transform para

In [18]:
# test_filled['newage'] = np.log(test_filled['age'])
# test_filled['newchldprsnt'] = np.sqrt(test_filled['chldprsnt'])

test_filled['newage'] = (test_filled['age'])
test_filled['newchldprsnt'] = (test_filled['chldprsnt'])

### Do one-hot encoding and split train and test

In [19]:
# one-hot encoding categorical predictors
predictors = ['gender', 'cd', 'hd', 'newage', 'dbdistance', 'vccdistance',
       'party', 'racename', 'hsonly', 'mrrg', 'newchldprsnt', 'cath', 'evang',
       'nonchrst', 'otherchrst', 'days.since.reg']
cate = ['gender', 'cd', 'hd', 'party', 'racename', ]

# predictors = ['gender', 'cd', 'hd', 'newage',
#        'party', 'racename', 'hsonly', 'mrrg', 'newchldprsnt','evang',
#        'nonchrst', 'otherchrst', 'days.since.reg']
# cate = ['gender', 'cd', 'hd', 'party', 'racename', ]

df_test_x=pd.DataFrame(test_filled[predictors])
df_test_x.head()

,gender,cd,hd,newage,dbdistance,vccdistance,party,racename,hsonly,mrrg,newchldprsnt,cath,evang,nonchrst,otherchrst,days.since.reg
0,M,2.0,52.0,30,2.296225,2.4741,L,Caucasian,19.5,21.2,25.3,9.8,16.6,45.2,28.4,393
1,F,5.0,19.0,20,2.296225,2.4741,U,Caucasian,39.7,20.2,29.1,12.0,14.4,41.4,32.2,668
2,M,4.0,44.0,56,2.296225,2.4741,R,Caucasian,11.3,62.7,41.3,14.8,14.7,36.0,34.6,606
3,F,7.0,34.0,20,2.296225,2.4741,R,Caucasian,32.8,11.6,33.1,14.5,10.3,44.6,30.6,565
4,F,6.0,41.0,26,2.296225,2.4741,D,Uncoded,10.2,14.7,22.4,8.2,18.4,43.5,29.9,336


In [20]:
# one hot encoding 'gender'
onehot = pd.get_dummies(df_test_x['gender'],prefix='gen')
df_test_x = df_test_x.drop('gender', axis=1)
df_test_x = df_test_x.join(onehot)

# one hot encoding 'cd'
onehot = pd.get_dummies(df_test_x['cd'],prefix='cd')
df_test_x = df_test_x.drop('cd', axis=1)
df_test_x = df_test_x.join(onehot)

# one hot encoding 'hd'
onehot = pd.get_dummies(df_test_x['hd'],prefix='hd')
df_test_x = df_test_x.drop('hd', axis=1)
df_test_x = df_test_x.join(onehot)

# one hot encoding 'party'
onehot = pd.get_dummies(df_test_x['party'],prefix='pty')
df_test_x = df_test_x.drop('party', axis=1)
df_test_x = df_test_x.join(onehot)

# one hot encoding 'racename'
onehot = pd.get_dummies(df_test_x['racename'])
df_test_x = df_test_x.drop('racename', axis=1)
df_test_x = df_test_x.join(onehot)

df_test_x.head(7)
print df_test_x.shape

(39510, 103)


In [21]:
# separate x
test_x=df_test_x.values
# print test_x[0:3,:]

### GridSearch by cross validation

In [22]:
# --------------
# A generic function to do CV
# --------------
def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds)
    gs.fit(X, y)

    best = gs.best_estimator_
    return best

In [23]:
def discrepancy(xx, yy):
    res = []
    for x, y in zip(xx, yy):
        res.append(-x*np.log(y)-(1-x)*np.log(1-y))
    return np.mean(res)

## Modeling and optimization

### Xgboost

In [24]:
my_y_train_BOOL=(my_y_train == 'Y')*1
my_y_test_BOOL=(my_y_test == 'Y')*1
dtrain = xgb.DMatrix(my_x_train, label = my_y_train_BOOL)
dtest = xgb.DMatrix(my_x_test, label = my_y_test_BOOL)

In [25]:

# param = {'max_depth':10, 'eta':0.15, 'silent':1, 
#          'objective':'binary:logistic',
#          'min_child_weight':2,'gamma':10 }  
param = {'max_depth':8, 'eta':0.15, 'silent':1, 
         'objective':'binary:logistic',
         'min_child_weight':2,'gamma':6,'nthread':4}  
watchlist  = [(dtest,'eval'), (dtrain,'train')]
num_round = 100

In [123]:
# start_time = time.time()
bst = xgb.train(param,dtrain,num_round,watchlist)
# compute_time = time.time() - start_time
# print compute_time

[0]	eval-error:0.30024	train-error:0.294996
[1]	eval-error:0.29592	train-error:0.292454
[2]	eval-error:0.2943	train-error:0.292589
[3]	eval-error:0.294604	train-error:0.290531
[4]	eval-error:0.293085	train-error:0.289316
[5]	eval-error:0.29403	train-error:0.288607
[6]	eval-error:0.29268	train-error:0.287122
[7]	eval-error:0.291297	train-error:0.284928
[8]	eval-error:0.291196	train-error:0.284613
[9]	eval-error:0.290959	train-error:0.283567
[10]	eval-error:0.29015	train-error:0.282622
[11]	eval-error:0.289778	train-error:0.281576
[12]	eval-error:0.289205	train-error:0.280564
[13]	eval-error:0.287787	train-error:0.27945
[14]	eval-error:0.287112	train-error:0.278528
[15]	eval-error:0.287416	train-error:0.278449
[16]	eval-error:0.287754	train-error:0.277628
[17]	eval-error:0.288293	train-error:0.276627
[18]	eval-error:0.28772	train-error:0.276368
[19]	eval-error:0.287247	train-error:0.275952
[20]	eval-error:0.286977	train-error:0.274928
[21]	eval-error:0.286674	train-error:0.274129
[22]	ev

In [124]:
preds=bst.predict(dtest)

In [125]:
labels = dtest.get_label()  
print ('error=%f' % ( sum(1 for i in range(len(preds)) if int(preds[i]>0.5)!=labels[i]) /float(len(preds))))  
print ('correct=%f' % ( sum(1 for i in range(len(preds)) if int(preds[i]>0.5)==labels[i]) /float(len(preds))))  

error=0.284615
correct=0.715385


In [126]:
discrepancy(labels, preds)

0.5564382309120639

In [ ]:
# choose parameters
max_depths = [6,8,10,12]
etas = [0.05,0.1,0.15,0.2]
min_child_weights = [1,2,4,6]
gammas=[2,6,10,12]

dis_old = 1
mmdd = 10
eett = 0.15
mmccww = 2
ggaa = 10

for md in max_depths:
    for et in etas:
        for mcw in min_child_weights:
            for ga in gammas:
                param['max_depth'] = md
                param['eta'] = et
                param['min_child_weight']=mcw
                param['gamma']=ga
                bst = xgb.train(param,dtrain,num_round)
                preds=bst.predict(dtest)
                dis = discrepancy(labels, preds)
                if dis < dis_old:
                    dis_old = dis
                    mmdd = md
                    eett = et
                    mmccww = mcw
                    ggaa = ga
                    print dis_old,mmdd,eett,mmccww,ggaa

### keras CNN

In [26]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using Theano backend.


In [27]:
print my_x_train.shape
print my_x_train[0][:15]

(88896, 103)
[   2.94443898    2.27622       2.444945     68.3           6.5
    6.43428318   15.8          22.2          29.6          32.4         293.
    1.            0.            0.            0.        ]


In [28]:
for i in range(15):
    print min(my_x_train[:,i]), max(my_x_train[:,i])

2.8903717579 4.61512051684
1.77177 45.9893
1.77177 45.9893
4.0 84.3
2.7 99.7
0.948683298051 9.5078914592
4.6 74.1
1.6 56.5
4.7 74.4
9.5 50.8
223.0 677.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0


In [29]:
ff_number = 11
tt_number = 103

In [30]:
my_x_train_temp = my_x_train[:,0:ff_number]
my_x_test_temp = my_x_test[:,0:ff_number]
my_x_train_temp[0]

array([   2.94443898,    2.27622   ,    2.444945  ,   68.3       ,
          6.5       ,    6.43428318,   15.8       ,   22.2       ,
         29.6       ,   32.4       ,  293.        ])

In [43]:
from sklearn import preprocessing
my_x_train_scale_temp = preprocessing.scale(my_x_train_temp, axis = 0)
my_x_train_scale = np.zeros((88896,tt_number))
my_x_train_scale[:,0:ff_number] = my_x_train_scale_temp
my_x_train_scale[:,ff_number:] = my_x_train[:,ff_number:]


my_x_test_scale_temp = preprocessing.scale(my_x_test_temp, axis = 0)
my_x_test_scale = np.zeros((29633,tt_number))
my_x_test_scale[:,0:ff_number] = my_x_test_scale_temp
my_x_test_scale[:,ff_number:] = my_x_test[:,ff_number:]


In [31]:
print my_x_train_scale[0]
print my_x_test_scale[0]

NameError: name 'my_x_train_scale' is not defined

In [32]:
quantiles = []
sample_num, fea_num = my_x_train_temp.shape
step = sample_num/99
for i in range(fea_num):
    tmp = sorted(my_x_train_temp[:,i])
    cur_row = step
    quant = []
    while cur_row < sample_num:
        quant.append(tmp[cur_row])
        cur_row += step
    quantiles.append(quant)
# print(len(quantiles[0]))

new_my_x_train_temp = my_x_train_temp.copy()
for fea_id in range(fea_num):
    for i in range(sample_num):
        val = my_x_train_temp[i, fea_id]
        num_greater = np.sum(val>quantiles[fea_id])
        new_my_x_train_temp[i, fea_id] = -0.4 + num_greater*0.01
print(new_my_x_train_temp)
    # my_x_train_scale_temp = preprocessing.scale(my_x_train_temp, axis = 0)

[[-0.4  -0.38 -0.38 ..., -0.27  0.24 -0.37]
 [ 0.46 -0.38 -0.38 ..., -0.4  -0.4   0.53]
 [-0.15 -0.38 -0.38 ...,  0.4  -0.05  0.54]
 ..., 
 [ 0.45 -0.38 -0.38 ...,  0.45 -0.17  0.4 ]
 [-0.21 -0.38 -0.38 ..., -0.28  0.1  -0.29]
 [-0.21 -0.38 -0.38 ...,  0.42 -0.39  0.5 ]]


In [33]:
# plt.figure()
# plt.hist(my_x_train[:,1], bins=10)
# # plt.hist(new_my_x_train_temp[:,0])
# plt.show()
# # print min(my_x_train[:,1]), max(my_x_train[:,1])
# # print(sorted(my_x_train_temp[:,1]))

In [34]:
np.mean(new_my_x_train_temp[:,0])

0.07464497840172786

In [35]:
my_x_train_scale = np.zeros((88896,tt_number))
my_x_train_scale.shape
# my_x_train_scale[:,0:11] = my_x_train_scale_temp
my_x_train_scale[:,0:ff_number] = new_my_x_train_temp
my_x_train_scale[:,ff_number:] = my_x_train[:,ff_number:]

In [36]:
### normalization
my_x_test_temp = my_x_test[:,0:ff_number]
# my_x_test_scale_temp = preprocessing.scale(my_x_test_temp, axis = 0)
new_my_x_test_temp = my_x_test_temp.copy()
for fea_id in range(new_my_x_test_temp.shape[1]):
    for i in range(new_my_x_test_temp.shape[0]):
        val = my_x_test_temp[i, fea_id]
        num_greater = np.sum(val>quantiles[fea_id])
        new_my_x_test_temp[i, fea_id] = -0.4 + num_greater*0.01

# print(my_x_test_temp)
my_x_test_scale = np.zeros((29633,tt_number))
# my_x_test_scale[:,0:11] = my_x_test_scale_temp
my_x_test_scale[:,0:ff_number] = new_my_x_test_temp
my_x_test_scale[:,ff_number:] = my_x_test[:,ff_number:]

In [37]:
my_x_test_scale[1]

array([ 0.56, -0.38, -0.38,  0.32,  0.16, -0.19,  0.46,  0.54, -0.36,
        0.28,  0.43,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ])

In [38]:
my_x_train_scale[1]

array([ 0.46, -0.38, -0.38,  0.17,  0.49, -0.18,  0.58,  0.18, -0.4 ,
       -0.4 ,  0.53,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ])

In [40]:
# Create first network with Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy
from keras import optimizers
# fix random seed for reproducibility
seed = 10
numpy.random.seed(seed)
model = Sequential()
model.add(Dense(128, input_dim=103, kernel_initializer='normal', activation='tanh'))
model.add(Dropout(0.2))
# model.add(Dense(128, kernel_initializer='normal', activation='tanh'))
# model.add(Dropout(0.2))
model.add(Dense(64, kernel_initializer='normal', activation='tanh'))
# model.add(Dense(16, kernel_initializer='normal', activation='relu'))
# model.add(Dense(4, kernel_initializer='normal', activation='relu'))
# model.add(Dense(4, kernel_initializer='normal', activation='relu'))
# model.add(Dense(10, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
# Compile model
# sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.5, nesterov=True)
model.compile(loss='binary_crossentropy', 
              optimizer="adam", 
              metrics=['accuracy'])
# Fit the model
model.fit(my_x_train_scale, my_y_train_BOOL, 
          validation_data=(my_x_test_scale, my_y_test_BOOL), 
          epochs=20, batch_size=32, verbose=2)


Train on 88896 samples, validate on 29633 samples
Epoch 1/20
2s - loss: 0.5756 - acc: 0.7034 - val_loss: 0.5694 - val_acc: 0.7099
Epoch 2/20
2s - loss: 0.5718 - acc: 0.7053 - val_loss: 0.5813 - val_acc: 0.7092
Epoch 3/20
2s - loss: 0.5712 - acc: 0.7053 - val_loss: 0.5685 - val_acc: 0.7110
Epoch 4/20
2s - loss: 0.5703 - acc: 0.7060 - val_loss: 0.5721 - val_acc: 0.7084
Epoch 5/20
2s - loss: 0.5693 - acc: 0.7067 - val_loss: 0.5684 - val_acc: 0.7088
Epoch 6/20
2s - loss: 0.5677 - acc: 0.7077 - val_loss: 0.5665 - val_acc: 0.7100
Epoch 7/20
2s - loss: 0.5661 - acc: 0.7079 - val_loss: 0.5638 - val_acc: 0.7115
Epoch 8/20
2s - loss: 0.5644 - acc: 0.7083 - val_loss: 0.5642 - val_acc: 0.7096
Epoch 9/20
2s - loss: 0.5621 - acc: 0.7102 - val_loss: 0.5648 - val_acc: 0.7110
Epoch 10/20
2s - loss: 0.5608 - acc: 0.7106 - val_loss: 0.5649 - val_acc: 0.7123
Epoch 11/20
2s - loss: 0.5605 - acc: 0.7109 - val_loss: 0.5628 - val_acc: 0.7121
Epoch 12/20
2s - loss: 0.5598 - acc: 0.7114 - val_loss: 0.5623 - val

In [49]:
# evaluate the model
scores = model.evaluate(my_x_train_scale, my_y_train_BOOL)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

88832/88896 [============================>.] - ETA: 0s

In [41]:
preds = model.predict(my_x_test_scale)
labels = my_y_test_BOOL 
print ('error=%f' % ( sum(1 for i in range(len(preds)) if int(preds[i]>0.5)!=labels[i]) /float(len(preds))))  
print ('correct=%f' % ( sum(1 for i in range(len(preds)) if int(preds[i]>0.5)==labels[i]) /float(len(preds))))  
discrepancy(labels, preds)

error=0.292276
correct=0.707724


0.56301671

In [73]:
# # reshape to be [samples][pixels][width][height]
# my_x_train_new = my_x_train[:,0:100]
# X_train_cnn = my_x_train_new.reshape(my_x_train_new.shape[0], 1, 10, 10).astype('float32')
# # X_test_cnn = X_test.reshape(X_test.shape[0], 1, 11, 10).astype('float32')

In [67]:
# import numpy
# from keras.datasets import mnist
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras.layers import Flatten
# from keras.layers.convolutional import Conv2D
# from keras.layers.convolutional import MaxPooling2D
# from keras.utils import np_utils

# # fix random seed for reproducibility
# seed = 7
# numpy.random.seed(seed)
# model = Sequential()
# # model.add(Dense(102, input_dim=102, kernel_initializer='uniform', activation='relu'))
# # model.add(Dense(10, kernel_initializer='uniform', activation='relu'))
# # model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
# model.add(Conv2D(32, (5,5), input_shape=(1,10,10), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# # model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(32,kernel_initializer='uniform',activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

# # Compile model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# # Fit the model
# model.fit(X_train_cnn, my_y_train_BOOL, epochs=30, batch_size=10,  verbose=2)


### Random forest

In [66]:
# --------------
# Optimize n_estimators, max_depth in random forest
# --------------
dt_rforest = ensemble.RandomForestClassifier(max_depth=3)
rforest = dt_rforest.fit(my_x_train, my_y_train)

parameters = {'max_depth': [12, 14, 16, 18, 20, 22, 24], 'n_estimators': [20, 25, 30, 35, 40]}
rforest = cv_optimize(rforest, parameters, my_x_train, my_y_train, n_jobs=10, n_folds=5, score_func=None)
print 'Optimal n_estimators: ' + str(rforest.n_estimators)
print 'Optimal max_depth: ' + str(rforest.max_depth)

Optimal n_estimators: 35
Optimal max_depth: 18


In [196]:
# --------------
# Random Forest
# --------------
start_time = time.time()
dt_rforest = ensemble.RandomForestClassifier(max_depth = 14, n_estimators = 100)
dt_rforest.fit(my_x_train, my_y_train)
compute_time = time.time() - start_time

print "Random forest score: " + str(dt_rforest.score(my_x_test, my_y_test))
print "Compute Time", compute_time

Random forest score: 0.713528836095
Compute Time 15.1984391212


In [197]:
dt_rforest.classes_

array(['N', 'Y'], dtype=object)

In [198]:
preds_rf = dt_rforest.predict_proba(my_x_test)

In [199]:
preds=preds_rf[:,1]

In [200]:
labels = my_y_test_BOOL 
print ('error=%f' % ( sum(1 for i in range(len(preds)) if int(preds[i]>0.5)!=labels[i]) /float(len(preds))))  
print ('correct=%f' % ( sum(1 for i in range(len(preds)) if int(preds[i]>0.5)==labels[i]) /float(len(preds))))  
discrepancy(labels, preds)

error=0.286471
correct=0.713529


0.56167985757320515

### KNN

In [86]:
# --------------
# Optimize n_neighbors in KNN
# --------------
dt_KNN = KNN(n_neighbors=5, weights='distance', n_jobs=10)
mKNN = dt_KNN.fit(my_x_train, my_y_train)

KNN_params = {'n_neighbors': [3, 5, 7, 9, 11, 13, 15, 17, 19, 21]}
mKNN = cv_optimize(mKNN, KNN_params, my_x_train, my_y_train, n_jobs=10, n_folds=5, score_func=None)
print 'Optimal n_neighbors: ' + str(mKNN.n_neighbors)

Optimal n_neighbors: 21


In [87]:
# --------------
# KNN
# --------------
start_time = time.time()
dt_KNN = KNN(n_neighbors=21, weights='distance', n_jobs=10)
mKNN = dt_KNN.fit(my_x_train, my_y_train)
compute_time = time.time() - start_time

print "KNN score: " + str(mKNN.score(my_x_test, my_y_test))
print "Compute Time", compute_time

KNN score: 0.676880504843
Compute Time 0.908000946045


In [88]:
dt_KNN.classes_

array(['N', 'Y'], dtype=object)

### GBoost

In [170]:
start_time = time.time()
mGBoost = GBoost()
mGBoost.fit(my_x_train, my_y_train)
compute_time = time.time() - start_time

print "GradBoosting score: " + str(mGBoost.score(my_x_test, my_y_test))
print "Compute Time", compute_time

GradBoosting score: 0.714406236291
Compute Time 34.2602109909


In [93]:
mGBoost.classes_

array(['N', 'Y'], dtype=object)

In [ ]:
discrepancy()

### Bagging

In [97]:
# --------------
# Optimize Bagging
# --------------
mBag = Bagging()
mBag.fit(my_x_train, my_y_train)
compute_time = time.time() - start_time

Bag_params = {'n_estimators': [10, 20, 30, 40, 50, 60]}
mBag = cv_optimize(mBag, Bag_params, my_x_train, my_y_train, n_jobs=10, n_folds=5, score_func=None)
print 'Optimal n_estimators: ' + str(mBag.n_estimators)

Optimal n_estimators: 60


In [98]:
start_time = time.time()
mBag = Bagging(n_estimators=60, n_jobs=10)
mBag.fit(my_x_train, my_y_train)
compute_time = time.time() - start_time

print "Bagging score: " + str(mBag.score(my_x_test, my_y_test))
print "Compute Time", compute_time

Bagging score: 0.702932541423
Compute Time 26.4830219746


In [99]:
mBag.classes_

array(['N', 'Y'], dtype=object)

### SVC

In [ ]:
# --------------
# Optimize SVM
# --------------
dt_SVM = SVC(C=50, kernel='linear')
mSVM = dt_SVM.fit(my_x_train, my_y_train)

SVM_params = {'C': [800, 1000, 1200, 1400, 1600]}
mSVM = cv_optimize(mSVM, SVM_params, my_x_train, my_y_train, n_jobs=10, n_folds=5, score_func=None)
print 'Optimal C: ' + str(mSVM.C)

In [ ]:
# --------------
# SVM
# --------------
start_time = time.time()
dt_SVM = SVC(C=1000, kernel='linear')
mSVM = dt_SVM.fit(my_x_train, my_y_train)
compute_time = time.time() - start_time

print "SVM score: " + str(mSVM.score(my_x_test, my_y_test))
print "Compute Time", compute_time

In [ ]:
mSVM.classes_

## Do prediction

### Xgboost

In [83]:
dtest_x = xgb.DMatrix(test_x)

In [84]:
pred_y = bst.predict(dtest_x)
print pred_y[0:5]
print np.shape(pred_y)

[ 0.58592987  0.36058328  0.74859029  0.55805337  0.78490901]
(39510,)


### Random Forest

In [154]:
pred_y = dt_rforest.predict_proba(test_x)
print pred_y[0:5,:]

[[ 0.44791708  0.55208292]
 [ 0.48620226  0.51379774]
 [ 0.24798737  0.75201263]
 [ 0.44153553  0.55846447]
 [ 0.24635485  0.75364515]]


### KNN

In [89]:
pred_y = dt_KNN.predict_proba(test_x)
print pred_y[0:5,:]

[[ 0.43069753  0.56930247]
 [ 0.4121044   0.5878956 ]
 [ 0.33823453  0.66176547]
 [ 0.43047334  0.56952666]
 [ 0.06307356  0.93692644]]


### GBoost

In [94]:
pred_y = mGBoost.predict_proba(test_x)
print pred_y[0:5,:]

[[ 0.33929182  0.66070818]
 [ 0.55804637  0.44195363]
 [ 0.27185358  0.72814642]
 [ 0.32067297  0.67932703]
 [ 0.20513272  0.79486728]]


### Bagging

In [100]:
pred_y = mBag.predict_proba(test_x)
print pred_y[0:5,:]

/Applications/anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/Applications/anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/Applications/anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...

[[ 0.5         0.5       ]
 [ 0.5         0.5       ]
 [ 0.38333333  0.61666667]
 [ 0.28333333  0.71666667]
 [ 0.11666667  0.88333333]]


## Save prediction

In [85]:
df_pred = pd.DataFrame(test_filled['Id'])
#df_pred['voted'] = pd.Series(pred_y[:,1], index=df_pred.index)
df_pred['voted'] = pd.Series(pred_y[:], index=df_pred.index)
df_pred.head()

,Id,voted
0,1,0.585930
1,2,0.360583
2,3,0.748590
3,4,0.558053
4,5,0.784909


In [86]:
#df_pred.to_csv('pred_RF_d18_n35.csv',index=False)
#df_pred.to_csv('pred_KNN_k21.csv',index=False)
#df_pred.to_csv('pred_GBoost.csv',index=False)
df_pred.to_csv('pred_xgboost3.csv',index=False)